# Define API keys and libraries

In [9]:
import json
import requests

In [ ]:
api_key = 'da56733a-6d62-4da1-9d2f-4e882d46478b'

# Define functions for calling API endpoints

In [84]:
class api_511:
    
    def __init__(self, api_key):
        self.api_key = api_key
    
    def get_request(self, url, params):
        response = requests.get(
            url = url,
            params = {
                'api_key': self.api_key,
                **params
            }
        )
        decoded = response.content.decode('utf-8-sig') # strip byte order mark from response
        return json.loads(decoded)
    
    def get_stops(self, operator_id='SF'):
        response = self.get_request(
            url = 'http://api.511.org/transit/stops',
            params = {
                'format': 'json',
                'operator_id': operator_id
            }
        )
        return response
    
    def get_stop_monitoring(self, route, stop_code, operator_id='SF'):
        response = self.get_request(
            url = 'http://api.511.org/transit/StopMonitoring',
            params = {
                'format': 'json',
                'agency': operator_id,
                'route': route,
                'stopCode': stop_code
            }
        )
        return response

In [85]:
api = api_511(api_key)

In [86]:
# List the stops available
stops = api.get_stops()

In [87]:
stops

{'Contents': {'ResponseTimestamp': '2023-01-17T17:47:27-08:00',
  'dataObjects': {'id': 'SF',
   'ScheduledStopPoint': [{'id': '15829',
     'Extensions': {'LocationType': None,
      'PlatformCode': None,
      'ParentStation': None},
     'Name': "100 O'Shaughnessy Blvd",
     'Location': {'Longitude': '-122.450678', 'Latitude': '37.744481'},
     'Url': 'https://www.sfmta.com/15829',
     'StopType': 'onstreetBus'},
    {'id': '13230',
     'Extensions': {'LocationType': None,
      'PlatformCode': None,
      'ParentStation': None},
     'Name': '10th Ave & Ortega St',
     'Location': {'Longitude': '-122.466502', 'Latitude': '37.752724'},
     'Url': 'https://www.sfmta.com/13230',
     'StopType': 'onstreetBus'},
    {'id': '13231',
     'Extensions': {'LocationType': None,
      'PlatformCode': None,
      'ParentStation': None},
     'Name': '10th Ave & Pacheco St',
     'Location': {'Longitude': '-122.466376', 'Latitude': '37.75086'},
     'Url': 'https://www.sfmta.com/13231',


In [76]:
# Get the live predictions for a stop
route = '48'
stop_code = '16121' # Portola Dr & Teresita Blvd
resp = api.get_stop_monitoring(route, stop_code)

In [77]:
resp

{'ServiceDelivery': {'ResponseTimestamp': '2023-01-18T01:43:11Z',
  'ProducerRef': 'SF',
  'Status': True,
  'StopMonitoringDelivery': {'version': '1.4',
   'ResponseTimestamp': '2023-01-18T01:43:11Z',
   'Status': True,
   'MonitoredStopVisit': [{'RecordedAtTime': '2023-01-18T01:43:14Z',
     'MonitoringRef': '16121',
     'MonitoredVehicleJourney': {'LineRef': '48',
      'DirectionRef': 'IB',
      'FramedVehicleJourneyRef': {'DataFrameRef': '2023-01-17',
       'DatedVehicleJourneyRef': '11082381_M11'},
      'PublishedLineName': 'QUINTARA-24TH STREET',
      'OperatorRef': 'SF',
      'OriginRef': '14781',
      'OriginName': 'Lower Great Hwy & Rivera St',
      'DestinationRef': '13411',
      'DestinationName': '3rd St + 20th St',
      'Monitored': True,
      'InCongestion': None,
      'VehicleLocation': {'Longitude': '-122.479439',
       'Latitude': '37.7484665'},
      'Bearing': '87.0000000000',
      'Occupancy': 'seatsAvailable',
      'VehicleRef': '8889',
      'Monit